In [10]:
"""Complex Networks Analysis Project.

* @File    :   Project.ipynb
* @Time    :   2025/04/05 18:02:17
* @Authors :   Marc Ballestero Ribó and Arnau Jutglar Puig
* @Version :   0
* @Contact :   marcballestero@ub.edu, ajutglpu7@alumnes.ub.edu
* @License :   GNU GPL v3.0
* @Desc    :   None
"""


'Complex Networks Analysis Project.\n\n* @File    :   Project.ipynb\n* @Time    :   2025/04/05 18:02:17\n* @Authors :   Marc Ballestero Ribó and Arnau Jutglar Puig\n* @Version :   0\n* @Contact :   marcballestero@ub.edu, ajutglpu7@alumnes.ub.edu\n* @License :   GNU GPL v3.0\n* @Desc    :   None\n'

# Complex Networks Analysis Project: A Study of the Air Transportation Multiplex Dataset

This notebook is devoted to presenting all the code and reasonings of our project for the Complex Networks Analysis course of the Master in Fundamental Principles of Data Science.

Here, we analyze the "Air Transportation Multiplex" dataset, which contains a multiplex network of airline routes among European airports, where each of the 37 edge types represents routes by a different airline. It is an undirected, multiplex and unweighted network.

The notebook is organized in sections each one corresponding to a working week/period. Here's a summary of all the work done:

- **`04/04/2025 → 11/04/2025:`**
- **`11/04/2025 → 25/04/2025:`**
- **`25/04/2025 → 09/05/2025:`**
- **`09/04/2025 → 16/05/2025:`**
- **`16/05/2025 → 23/05/2025:`**
- **`23/05/2025 → 30/05/2025:`**

***

### Necessary Imports and Directory Management


In [11]:
# Reset the kernel
%reset -f

# Necessary imports
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from pathlib import Path


In [12]:
# Directories
DATA_DIR = Path("../Data")


***

## **04/04/2025 → 11/04/2025**

This week, we have focused our efforts on understanding the format of our dataset, loading the network and doing some basic exploration of it.

### Data Reading

In [13]:
# Read the file containing the airport information
df_airports = pd.read_csv(
    DATA_DIR / "airports.tsv",
    sep="\t",
    header=None,
    names=["node_id", "icao_code", "longitude", "latitude"],
)


In [ ]:
# Read the file containing the airport connections
def read_multilayer_connections(
    file_path: Path,
) -> list[dict]:
    """Reads the multilayer connections from a file.

    Args:
        file_path (Path) -- Path to the file containing the multilayer connections.

    Returns:
        list -- List containing the multilayer connections.
    """
    layers = []

    with Path.open(file_path, "r") as f_in:
        lines = [line.strip() for line in f_in if line.strip()]

    i = 0
    while i < len(lines):
        num_nodes = int(lines[i])
        i += 1

        edges = set()
        for _ in range(num_nodes):
            parts = lines[i].split()
            node_id = int(parts[0])
            neighbors = list(map(int, parts[2:]))
            for neighbor in neighbors:
                edge = tuple(sorted((node_id, neighbor)))  # Undirected
                edges.add(edge)
            i += 1

        layers.append({
            "active_nodes": num_nodes,
            "edges": edges
        })

    return layers


In [ ]:
# Dummy file contents for testing

layers = read_multilayer_connections(DATA_DIR / "network.txt")

# Create a graph for each layer
graphs = []
for layer in layers:
    G = nx.Graph()
    G.add_edges_from(layer["edges"])
    graphs.append(G)



In [ ]:
# Function to plot the graph as an interactive map
def plot_interactive_map(
    node_df: pd.DataFrame,
    edge_list: list[tuple[int, int]],
) -> folium.Map:
    """Plots an interactive map with nodes and edges.
    Args:
        node_df (pd.DataFrame) -- DataFrame containing node information.
        edge_list (list[tuple[int, int]]) -- List of edges.
    Returns:
        folium.Map -- Interactive map with nodes and edges.
    """
    center = [node_df["latitude"].mean(), node_df["longitude"].mean()]
    m = folium.Map(location=center, zoom_start=5)

    # Add nodes
    for _, row in node_df.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=3,
            color="red",
            fill=True,
            fill_color="red"
        ).add_to(m)

    # Add edges
    for n1, n2 in edge_list:
        coord1 = node_df[node_df["node_id"] == n1][["latitude", "longitude"]].values[0]
        coord2 = node_df[node_df["node_id"] == n2][["latitude", "longitude"]].values[0]
        folium.PolyLine(locations=[coord1, coord2], color="blue", weight=1).add_to(m)

    return m


In [20]:
m = plot_interactive_map(
    node_df=df_airports,
    edge_list=layers[10]['edges'],
)
m
